In [1]:
import requests
import re
from collections import OrderedDict
from itertools import repeat
from tqdm import tqdm_notebook
import pandas as pd
import lxml.html
from selenium.webdriver import Chrome

In [3]:
data = pd.read_csv('poetry_url.csv')

In [56]:
data.head()

,url,title,writer
0,http://poemlove.co.kr/m/bbs/board.php?bo_table...,동시/첫눈 오는 날.,장수남
1,http://poemlove.co.kr/m/bbs/board.php?bo_table...,"그리움, 너는",고은영
2,http://poemlove.co.kr/m/bbs/board.php?bo_table...,비워내는 건 외로운 법이다,고은영
3,http://poemlove.co.kr/m/bbs/board.php?bo_table...,지극히 따뜻한 애혼(愛魂)에게,고은영
4,http://poemlove.co.kr/m/bbs/board.php?bo_table...,12월엔 설화(雪花)를 그리는 한 마리 노루이고 싶다,고은영


In [5]:
urls = data.url

In [33]:
urls[10]

'http://poemlove.co.kr/m/bbs/board.php?bo_table=tb01&wr_id=254754'

In [49]:
req = requests.get(urls[0])
root = lxml.html.fromstring(req.text)

In [47]:
text = root.cssselect('span#writeContents')[0].text_content()
text = re.sub("[(\xa0)]", "", text)
text = re.sub('[(\u3000)]',' ', text)
text = re.sub("[a-zA-Z]+ *?[a-zA-Z]+", "<ENG>", text)
text = re.sub("[0-9]+ *?[0-9]+", "<NUM>", text)
[x.strip() for x in re.split(r"[\r]", text) if len(x.strip()) > 1]

['대물림',
 '목필균',
 '배구선수가 어머니가 되고',
 '딸이 배구선수가 되고',
 '응원을 해 주던 아이가',
 '수없이 코트에 뒹굴어 선수가 되어',
 '엄마의 응원을 받고',
 '이겨도, 져도 졸리는 심장',
 '국가대표로 가는 길이 어땠을까',
 '부상과 회복을 거듭하면서도',
 '코트에서 행복한 것을 물려받는다',
 '딸로 태어나서, 어머니가 되고',
 '시집간 딸이 자식을 기르고',
 '삶의 희로애락이던 아이가',
 '수없이 일상에 뒹굴어 어른이 되어',
 '자식 때문에 웃고 우는 엄마가 되고',
 '잘 되어도, 일탈을 해도 근심으로 산다',
 '어머니로 가는 길이 비단길일끼',
 '사랑과 질책을 거듭하면서도',
 '딸의 자식에게 행복이라 한다']

In [30]:
urls[:100][]

'http://poemlove.co.kr/m/bbs/board.php?bo_table=tb01&wr_id=254765'

In [74]:
contents = []
for i in tqdm_notebook(urls):
    try:
        req = requests.get(i)
        root = lxml.html.fromstring(req.text)
        text = root.cssselect('span#writeContents')[0].text_content()
        text = re.sub("[(\xa0)]", "", text)
        text = re.sub('[(\u3000)]',' ', text)
        text = re.sub("[a-zA-Z]+ *?[a-zA-Z]+", "<ENG>", text)
        text = re.sub("[0-9]+ *?[0-9]+", "<NUM>", text)
        text = re.sub("[一-龥]+", "<CHI>", text)
        text = re.sub("[ぁ-ゔ]+|[ァ-ヴー]+", "<JAP>", text)
        contents.append([x.strip() for x in re.split(r"[\r]", text) if len(x.strip()) > 1])
    except:
        contents.append(None)

In [75]:
data['contents'] = contents

In [76]:
data.to_csv('poetry.csv',encoding='UTF8',index=False)

In [77]:
pd.read_csv('poetry.csv')

,url,title,writer,contents
0,http://poemlove.co.kr/m/bbs/board.php?bo_table...,동시/첫눈 오는 날.,장수남,"['동시/첫눈 오는 날.', '은빛하늘 하얀 꿈', '하얀 둥지 틀고', '아장아장..."
1,http://poemlove.co.kr/m/bbs/board.php?bo_table...,"그리움, 너는",고은영,"['그리움, 너는 / <CHI>고은영', '비 오는 저 들녘', '눈 내리는 어둑한..."
2,http://poemlove.co.kr/m/bbs/board.php?bo_table...,비워내는 건 외로운 법이다,고은영,"['비워내는 건 외로운 법이다 / <CHI>고은영', '깊은 겨울 추위 속에도 창가..."
3,http://poemlove.co.kr/m/bbs/board.php?bo_table...,지극히 따뜻한 애혼(愛魂)에게,고은영,"['지극히 따뜻한 애혼<CHI>에게 / <CHI>고은영', '너의 영혼에 나를 심고..."
4,http://poemlove.co.kr/m/bbs/board.php?bo_table...,12월엔 설화(雪花)를 그리는 한 마리 노루이고 싶다,고은영,['<NUM>월엔 설화<CHI>를 그리는 한 마리 노루이고 싶다 / <CHI>고은영...
5,http://poemlove.co.kr/m/bbs/board.php?bo_table...,감잎을 쓸어 모으며,김동기,"['감잎을 쓸어 모으며', '가을이란 이름으로', '이 땅에 내려와 나뒹굴던', '..."
6,http://poemlove.co.kr/m/bbs/board.php?bo_table...,겨울행 열차 - 문학저널,임영준,"['겨울행 열차', '과연', '우리 중에 누가', '홀가분한 왕복표를', '끊을 ..."
7,http://poemlove.co.kr/m/bbs/board.php?bo_table...,착각,이남일,"['착각', '이남일', '나는 나를 볼 수 없고', '너는 너를 볼 수 없다.',..."
8,http://poemlove.co.kr/m/bbs/board.php?bo_table...,시인에게,이문자,"['아름다운 바보', '동시사랑 이문자', '세상 모든', '아픔과 슬픔까지도',..."
9,http://poemlove.co.kr/m/bbs/board.php?bo_table...,수선화,이문자,"['수선화', '동시사랑 이문자', '수선이 뾰족히', '솟아 나왔네', '올 해..."
